# Gender Bias Recruitment - Simulator Candidates

Este notebook permite inserir os dados de um candidato individualmente e verificar a **chance de aprovação** segundo o modelo treinado.  
Ele foi desenvolvido para testar **equidade de gênero**: você pode comparar candidatos com os mesmos atributos físicos e testes, mudando apenas o gênero, para observar se o modelo mantém a mesma chance de aprovação.

## 1. Variáveis de Entrada

- **Idade (Age_feature_sensitive)**: idade do candidato.  
- **Força (Strength_hidden)**: medida de força física do candidato.  
- **Velocidade (Speed_hidden)**: medida de velocidade física.  
- **Teste de Velocidade (Speedtest_hidden)**: resultado do teste de velocidade.  
- **Teste de Força (Lifttest_hidden)**: resultado do teste de força.  
- **Gênero (Gender_feature_sensitive)**: gênero do candidato (`male` ou `female`).  

## 2. Objetivos

- Permitir **simulação de candidatos individuais**.  
- Avaliar a **probabilidade de aprovação** segundo o modelo treinado.  
- Comparar candidatos para verificar **fairness do modelo** entre homens e mulheres.

In [54]:
import pandas as pd
import numpy as np
import pickle

from IPython.display import display

def load_object(path):
    with open(path, "rb") as f:
        return pickle.load(f)

model_fair = load_object("../outputs/model_rf_fair.pkl")


In [55]:
print("Preencha as informações do candidato:")

# Características sensíveis
idade = int(input("Idade do candidato: "))
genero = input("Gênero do candidato (male/female): ").strip().lower()

# Características de teste do candidato
velocidade = float(input("Velocidade de reação (Speed_hidden, ex: 0-2): "))
forca = float(input("Força (Strength_hidden, ex: 0-6): "))
teste_velocidade = float(input("Resultado do teste de velocidade (Speedtest_hidden, 0-1): "))
teste_forca = float(input("Resultado do teste de força (Lifttest_hidden, 0-1): "))
teste_resultado = float(input("Resultado do teste geral (testresult_feature, 0-2): "))

# Valores econômicos/derivados
valor_quando_contratado = float(input("Valor que o candidato gera se contratado (Value-when-hired_hidden, euros): "))
perda_quando_nao_contratado = float(input("Valor perdido se não contratado (Candidate-loss-when-nothired_hidden, euros): "))

df_candidato = pd.DataFrame({
    # IDs ocultos
    "mainid_hidden": [0],
    "mod15_hidden": [0],
    "div15_hidden": [0],
    
    # Testes e atributos
    "Speed_hidden": [velocidade],
    "Strength_hidden": [forca],
    "Speedtest_hidden": [teste_velocidade],
    "Lifttest_hidden": [teste_forca],
    "testresult_feature": [teste_resultado],
    
    # Variáveis auxiliares
    "livesnear_feature": [1],
    "Suitability_target": [1],
    "Value-when-hired_hidden": [valor_quando_contratado],
    "Candidate-loss-when-nothired_hidden": [perda_quando_nao_contratado],
    "Shouldnothire": [0],
    "hired-by-expert": [1],
    "A1_testresult": [1],
    "A2_testresult_30under": [1],
    "A3_Age_Gender_test": [idade],
    "A4_postive-dicr": [1], 
    
    # Gênero (sensível)
    "Gender_feature_sensitive": [genero]
})

print("\nDados do candidato:")
display(df_candidato)

Preencha as informações do candidato:

Dados do candidato:


,mainid_hidden,mod15_hidden,div15_hidden,Speed_hidden,Strength_hidden,Speedtest_hidden,Lifttest_hidden,testresult_feature,livesnear_feature,Suitability_target,Value-when-hired_hidden,Candidate-loss-when-nothired_hidden,Shouldnothire,hired-by-expert,A1_testresult,A2_testresult_30under,A3_Age_Gender_test,A4_postive-dicr,Gender_feature_sensitive
0,0,0,0,10.0,10.0,10.0,10.0,10.0,1,1,10.0,10.0,0,1,1,1,20,1,male


In [56]:
colunas_modelo = [
    "mainid_hidden", "mod15_hidden", "div15_hidden", "Speed_hidden", "Strength_hidden",
    "Speedtest_hidden", "Lifttest_hidden", "testresult_feature", "livesnear_feature",
    "Suitability_target", "Value-when-hired_hidden", "Candidate-loss-when-nothired_hidden",
    "Shouldnothire", "hired-by-expert", "A1_testresult", "A2_testresult_30under",
    "A3_Age_Gender_test", "A4_postive-dicr"
]

df_candidato_modelo = pd.DataFrame({
    col: [df_candidato[col][0]] if col in df_candidato.columns else [0]
    for col in colunas_modelo
})

proba = model_fair.predict_proba(df_candidato_modelo)[:,1][0]
resultado = model_fair.predict(df_candidato_modelo)[0]

print(f"\nProbabilidade de aprovação: {proba*100:.2f}%")
print(f"Previsão final do modelo: {'Aprovado' if resultado==1 else 'Não aprovado'}")


Probabilidade de aprovação: 57.00%
Previsão final do modelo: Aprovado
